# Description

Notebook for loading simulation results from updated neff experiments.

# 1. Imports

In [112]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import multiprocessing
import numpy as np
import os
import pandas as pd
import pingouin as pg
import pickle
import seaborn as sns
import sys

from tqdm import tqdm

# user imports
sys.path.append("../../")

from utils.pwr import *

# 2. Load simulated results data

## Simulation constants

In [140]:
n_trials = 500
fuzzy_gaps = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
alpha = 0.05

low_cutoff = 0.25
hi_cutoff = 0.75

seeds = range(0, 401, 100)

In [141]:
RESULTS_DIR = "../../results/kdd23/"

## Baseline discovery

In [142]:
baseline_result_dict = {}

for fuzzy_gap in tqdm(fuzzy_gaps):
    error_dict = {
        'x': {
            'fp': 0,
            'lower_fn': 0,
            'upper_fn': 0
        },

        'covar': {
            'fp': 0
        },
    }
    pvals = {}
    for seed in seeds:
        with open(f"../../experiments/kdd/baseline_discovery/seed{seed}/blended_rdd_fixed_bw_{fuzzy_gap}.pkl", "rb") as f:
            results = pickle.load(f)
            for result in results:
                x_thresholds = [np.round(c, decimals=2) for c in result['x'].keys()]
                if low_cutoff not in x_thresholds:
                    error_dict['x']['lower_fn'] += 1
                else: 
                    x_thresholds.remove(0.25)

                if hi_cutoff not in x_thresholds:
                    error_dict['x']['upper_fn'] += 1
                else:
                    x_thresholds.remove(0.75)

                error_dict['x']['fp'] += len(x_thresholds)
                error_dict['covar']['fp'] += len(result['covar'].keys())

            
    baseline_result_dict[fuzzy_gap] = error_dict       


100%|██████████| 6/6 [00:11<00:00,  1.93s/it]


In [143]:
pickle.dump(baseline_result_dict, open(os.path.join(RESULTS_DIR, "blend_baseline_results.dict"), "wb"))

## Subgroup discovery

In [193]:
sim_dir = "/data/REDACTED/rdsgd/subgroup/"

In [194]:
def process_subgroup_tree_results(fuzzy_gap, seeds):
    """Process subgroup tree results"""
    pvals = {
        'trial': [],
        'cutoff': [],
        'tau_pval': [],
        'neff_pval': [],
        'rule_length': [],
    }
    for seed in seeds:
        with open(os.path.join(sim_dir, f"seed{seed}_blended_rdd_fixed_bw_{fuzzy_gap}.pkl"), "rb") as f:
            result, n_tests = pickle.load(f)    
            
            x_dict = result['x']
            x_thresholds = [np.round(c, decimals=2) for c in x_dict.keys()]
            
            for x_cutoff in x_thresholds:
                nodes = x_dict[x_cutoff]
                
                for node in nodes:
                    if node['llr_results'] is None:
                        continue
                
                    pvals['trial'].append(seed)
                    pvals['cutoff'].append(x_cutoff)
                    pvals['neff_pval'].append(node['neff_pval'])
                    pvals['tau_pval'].append(node['llr_results'].pvalues['z'])
                    pvals['rule_length'].append(len(node['rule_path']))
                    #pvals[seed]['x_all'][x_cutoff].append((node['llr_results'].pvalues['z'], node['neff_pval'], len(node['rule_path'])))
    
    pval_df = pd.DataFrame.from_dict(pvals)
    pval_df['fuzzy_gap'] = fuzzy_gap
    return pval_df

In [195]:
%%time
#subgroup_results_dict = {}

seeds = range(0, n_trials)

f_args = [(fuzzy_gap, seeds) for fuzzy_gap in fuzzy_gaps]
with multiprocessing.Pool(8) as p:
    results = p.starmap(process_subgroup_tree_results, f_args)

#subgroup_results_dict = {fuzzy_gap: pvals for fuzzy_gap, pvals in results}
pval_df = pd.concat(results)

invalid value encountered in sqrt
invalid value encountered in sqrt
invalid value encountered in sqrt
invalid value encountered in sqrt
invalid value encountered in sqrt
invalid value encountered in sqrt


CPU times: user 93.7 ms, sys: 216 ms, total: 310 ms
Wall time: 1min 3s


In [196]:
pval_df

,trial,cutoff,tau_pval,neff_pval,rule_length,fuzzy_gap
0,0,0.05,0.249188,NaN,1,0.2
1,0,0.05,0.018973,1.259842e+00,2,0.2
2,0,0.05,0.777216,9.696829e-01,2,0.2
3,0,0.10,0.607564,NaN,1,0.2
4,0,0.15,0.323746,NaN,1,0.2
...,...,...,...,...,...,...
17033,499,0.85,0.293436,NaN,1,0.7
17034,499,0.85,0.028985,1.983412e+00,2,0.7
17035,499,0.85,0.837292,8.638401e-11,2,0.7
17036,499,0.90,0.081328,NaN,1,0.7


In [197]:
pval_df.to_pickle(os.path.join(RESULTS_DIR, "subgroup_neff_pval.df"))

## Compute corrected p-values

In [198]:
alpha = 0.05
low_cutoff = 0.25
upper_cutoff = 0.75

#tp_all = {}

nb_dict = {}

subgroup_trial_dict = {}

#pval_df = result_dict['x_all']
#pval_df.shape
for gap in tqdm(fuzzy_gaps):
    gap_dict = {
        'upper_tp': 0,
        'lower_tp': 0,
        'fp': 0,
        'tot_tests': 0,
        'tot_upper_tp': 0,
        'tot_lower_tp': 0,
        #'fp_cutoffs': set()
    }
    
    for trial in range(n_trials):
        trial_df = pval_df[(pval_df['trial'] == trial) & (pval_df['fuzzy_gap'] == gap)]
        gap_dict['tot_tests'] += trial_df.shape[0]
        
        method = 'bonf'
        #reject_neff, _ = pg.multicomp(list(trial_df['neff_pval']), method=method, alpha=alpha)
        reject_neff = trial_df['neff_pval'] < alpha
        reject_z, _ = pg.multicomp(list(trial_df['tau_pval']), method=method, alpha=alpha)

        reject = (reject_z & reject_neff) | (reject_z & (trial_df['rule_length'] == 1))
        if not hasattr(reject, '__iter__'):
            reject = [reject]
        sig_df = trial_df[reject]
        
        if low_cutoff in list(sig_df['cutoff']):
            gap_dict['lower_tp'] += 1
            gap_dict['tot_lower_tp'] += sig_df[sig_df['cutoff'] == low_cutoff].shape[0]

        if upper_cutoff in list(sig_df['cutoff']):
            gap_dict['upper_tp'] += 1
            gap_dict['tot_upper_tp'] += sig_df[sig_df['cutoff'] == upper_cutoff].shape[0]
            
        # remaining sig values are false positives
        fp_df = sig_df[~sig_df['cutoff'].isin([low_cutoff, upper_cutoff])]
        #print(fp_df)
        gap_dict['fp'] += fp_df.shape[0]
        # for x in fp_df['cutoff'].unique():
        #     gap_dict['fp_cutoffs'].add(x)
        
    nb_dict[gap] = gap_dict
    #subgroup_trial_dict[gap] = trial_gap_dict
        

100%|██████████| 6/6 [00:06<00:00,  1.12s/it]


In [199]:
nb_dict

{0.2: {'upper_tp': 32,
  'lower_tp': 47,
  'fp': 54,
  'tot_tests': 18444,
  'tot_upper_tp': 33,
  'tot_lower_tp': 50},
 0.3: {'upper_tp': 116,
  'lower_tp': 125,
  'fp': 68,
  'tot_tests': 18768,
  'tot_upper_tp': 121,
  'tot_lower_tp': 145},
 0.4: {'upper_tp': 269,
  'lower_tp': 284,
  'fp': 114,
  'tot_tests': 18384,
  'tot_upper_tp': 312,
  'tot_lower_tp': 359},
 0.5: {'upper_tp': 440,
  'lower_tp': 420,
  'fp': 158,
  'tot_tests': 17956,
  'tot_upper_tp': 619,
  'tot_lower_tp': 613},
 0.6: {'upper_tp': 496,
  'lower_tp': 481,
  'fp': 291,
  'tot_tests': 17582,
  'tot_upper_tp': 862,
  'tot_lower_tp': 797},
 0.7: {'upper_tp': 500,
  'lower_tp': 498,
  'fp': 497,
  'tot_tests': 17038,
  'tot_upper_tp': 973,
  'tot_lower_tp': 900}}

In [200]:
pickle.dump(nb_dict, open(os.path.join(RESULTS_DIR, "rdsgd_neff_results.dict"), "wb"), -1)